<a href="https://colab.research.google.com/github/SriSatyaLokesh/some-app-for-whatsapp/blob/master/notebooks/analytics/Visualization_Vasavi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()

Saving WhatsApp Chat with Priyanka_VL.txt to WhatsApp Chat with Priyanka_VL.txt


# DataFrame

In [6]:
import pandas as pd
from itertools import chain, repeat
import re

chainer = chain.from_iterable
class Cleaner:
  def __init__(self,filename):
    self.filename = filename

  def _clean_data(self):
    chat_file = self.filename
    with open(chat_file) as file:
      all_msgs = file.readlines()
      raw_texts = []
      for line in all_msgs:
        if re.match("^\d{2}/\d{2}/\d{2,4},\s\d{1,2}:\d{2}\s[ap]*[m]*",line):
          date_split = line.split(' - ') #date and user
          user_split = date_split[1].split(':')
          if len(user_split) >= 2:
            raw_texts.append(line)
        else:
          try:
            raw_texts[-1] += line
          except Exception as err:
            # print(line,raw_texts)
            # print(err)
            pass

    return raw_texts 
      
  def _get_data(self):
    raw_texts = self._clean_data()
    raw_messages = dict()
    for line in raw_texts:
      date_split = line.split(' - ')
      user_split = date_split[1].split(':')
      user = user_split[0]
      if user not in raw_messages:
        raw_messages[user] = [line]
      else:
        raw_messages[user].append(line)

    df = pd.DataFrame({'user': list(chainer(repeat(k, len(v)) for k,v in raw_messages.items())),
                   'raw_text': list(chainer(raw_messages.values()))}) 
    return df

chat_file = "WhatsApp Chat with Priyanka_VL.txt"
clean = Cleaner(chat_file)
df = clean._get_data()
df

,user,raw_text
0,Priyanka_VL,"19/09/19, 10:48 am - Priyanka_VL: Hii\n"
1,Priyanka_VL,"19/09/19, 10:48 am - Priyanka_VL: Am in office\n"
2,Priyanka_VL,"19/09/19, 10:48 am - Priyanka_VL: Niku anta ti..."
3,Priyanka_VL,"19/09/19, 10:49 am - Priyanka_VL: Priyanka\n"
4,Priyanka_VL,"19/09/19, 10:49 am - Priyanka_VL: Ok\n"
...,...,...
1066,Vasavi,"26/03/20, 9:03 pm - Vasavi: Ma intlo tv undhi ..."
1067,Vasavi,"26/03/20, 9:04 pm - Vasavi: Hmmmmmmmmmmmnnn\n"
1068,Vasavi,"26/03/20, 9:05 pm - Vasavi: 👋🏻\n"
1069,Vasavi,"24/04/20, 11:31 pm - Vasavi: Intern letter vac..."


In [7]:
from string import digits
df["raw_message"] = df.apply(lambda df : "".join(df['raw_text'].split(":")[2:]),axis=1)
df["raw_message"] = df.apply(lambda df : df['raw_message'][:-1],axis=1)
remove_digits = str.maketrans('', '', digits)
df["text_only_message"] = df.apply(lambda df : df["raw_message"].encode('ascii', 'ignore').decode('ascii'),axis=1)
df["text_only_message"] = df.apply(lambda df : df["text_only_message"].translate(remove_digits),axis=1)
df.loc[(df['text_only_message'] == " This message was deleted") | (df['text_only_message'] == " <Media omitted>"),'text_only_message'] = ""
df = df.groupby('user').filter(lambda x : len(x) > 10)

df['date'] =  df.apply(lambda df : df['raw_text'].split(" - ")[0],axis=1)

temp = ["%d/%m/%Y, %I:%M %p" , "%d/%m/%y, %I:%M %p" , "%d/%m/%Y, %H:%M" , "%d/%m/%y, %H:%M" ,
        "%d/%Y/%m, %I:%M %p" , "%d/%y/%m, %I:%M %p" , "%d/%Y/%m, %H:%M" , "%d/%y/%m, %H:%M" ,
        "%Y/%m/%d, %I:%M %p" , "%y/%m/%d, %I:%M %p" , "%Y/%m/%d, %H:%M" , "%y/%m/%d, %H:%M" ,
        "%Y/%d/%m, %I:%M %p" , "%y/%d/%m, %I:%M %p" , "%Y/%d/%m, %H:%M" , "%y/%d/%m, %H:%M" ,
        "%m/%Y/%d, %I:%M %p" , "%m/%y/%d, %I:%M %p" , "%m/%Y/%d, %H:%M" , "%m/%y/%d, %H:%M" ,
        "%m/%d/%Y, %I:%M %p" , "%m/%d/%y, %I:%M %p" , "%m/%d/%Y, %H:%M" , "%m/%d/%y, %H:%M"]

for formats in temp:       
  try:
    df['date'] = pd.to_datetime(df['date'], format=formats)
  except:
    continue

df['hour'] = df['date'].dt.hour
df['weekday'] = df['date'].dt.weekday
df

,user,raw_text,raw_message,text_only_message,date,hour,weekday
0,Priyanka_VL,"19/09/19, 10:48 am - Priyanka_VL: Hii\n",Hii,Hii,2019-09-19 10:48:00,10,3
1,Priyanka_VL,"19/09/19, 10:48 am - Priyanka_VL: Am in office\n",Am in office,Am in office,2019-09-19 10:48:00,10,3
2,Priyanka_VL,"19/09/19, 10:48 am - Priyanka_VL: Niku anta ti...",Niku anta time partadi,Niku anta time partadi,2019-09-19 10:48:00,10,3
3,Priyanka_VL,"19/09/19, 10:49 am - Priyanka_VL: Priyanka\n",Priyanka,Priyanka,2019-09-19 10:49:00,10,3
4,Priyanka_VL,"19/09/19, 10:49 am - Priyanka_VL: Ok\n",Ok,Ok,2019-09-19 10:49:00,10,3
...,...,...,...,...,...,...,...
1066,Vasavi,"26/03/20, 9:03 pm - Vasavi: Ma intlo tv undhi ...",Ma intlo tv undhi chusthune una🤦‍♀\nWarangal ...,Ma intlo tv undhi chusthune una\nWarangal lo ...,2020-03-26 21:03:00,21,3
1067,Vasavi,"26/03/20, 9:04 pm - Vasavi: Hmmmmmmmmmmmnnn\n",Hmmmmmmmmmmmnnn,Hmmmmmmmmmmmnnn,2020-03-26 21:04:00,21,3
1068,Vasavi,"26/03/20, 9:05 pm - Vasavi: 👋🏻\n",👋🏻,,2020-03-26 21:05:00,21,3
1069,Vasavi,"24/04/20, 11:31 pm - Vasavi: Intern letter vac...",Intern letter vachindha,Intern letter vachindha,2020-04-24 23:31:00,23,4


# Weekday Graph

In [0]:
import plotly.express as px